Turns out Microsoft Edge doesn't really work on mac. Who knew. Look into remote driver later

In [46]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, random

In [51]:
word_bank = './word_bank.csv'

In [52]:
df = pd.read_csv(word_bank)

In [53]:
df.head()

,posts
0,CNN's Anthony Bourdain dead at 61
1,Senate Approves Marijuana Legalization Bill in...
2,'No Gays Allowed': Tennessee store owner puts ...
3,"Suicide rates are up 30 percent since 1999, CD..."
4,Video shows police hitting homeless man as he ...


In [54]:
# Loading Webpage
driver = webdriver.Chrome()

driver.set_window_size(1280, 720)
driver.get('https://www.bing.com/')

In [55]:
# Logging into account

In [56]:
# begining search loop

driver.find_element_by_xpath('//*[@id="sb_form_q"]').clear()
element = driver.find_element_by_xpath('//*[@id="sb_form_q"]')
element.send_keys('testing')
time.sleep(1)
element.send_keys(Keys.ENTER)
time.sleep(1)

In [64]:
indx = 0
# element = driver.find_element_by_xpath('//*[@id="sb_form_1"]')
for i in range(len(df)):
    element.send_keys(df.posts[indx])
    indx += 1
    

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=66.0.3359.181)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.3 x86_64)


In [75]:
for i in range(df):
    df.posts.iloc(i, df.posts)

TypeError: 'DataFrame' object cannot be interpreted as an integer

In [84]:
num=0
df.shape[0]
# for i in df):
#     print(df.posts(num))
#     num += i

90